### Import dependencies


In [15]:


from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PayloadSchemaType, PointStruct, SparseVectorParams, Document, Prefetch, FusionQuery
from qdrant_client import models

import pandas as pd
import openai
import cohere

from dotenv import load_dotenv

import os



### Retrieval

In [16]:
load_dotenv()
qdrant_client = QdrantClient(url="http://localhost:6333")

In [4]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=[text],
        model=model,
    )
    return response.data[0].embedding

In [5]:
def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-01-hybrid-search",
        prefetch=[
            Prefetch(
                query=query_embedding,
                using="text-embedding-3-small",
                limit=20
            ),
            Prefetch(
                query=Document(
                    text=query,
                    model="qdrant/bm25"
                ),
                using="bm25",
                limit=20
            )
        ],
        query=FusionQuery(fusion="rrf"),
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    similarity_scores = []
    retrieved_context_ratings = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


In [6]:


query = "Can I get a laptop?"



In [7]:

results = retrieve_data(query, qdrant_client, k=20)

In [8]:

results

{'retrieved_context_ids': ['B0CGRSBNY1',
  'B0BVZ47N6N',
  'B09W9DV8PX',
  'B087ZMF5PR',
  'B0B87VXYFR',
  'B0BQFJVQ15',
  'B0944QNV3W',
  'B0B7XC9VL4',
  'B0BTKWY6XG',
  'B0C3RCQDM9',
  'B09QS7W8G5',
  'B0B8VH5WBR',
  'B0B46D4FM3',
  'B09T5YML6T',
  'B015TPLFFO',
  'B0CFLNNB4V',
  'B0BSGDCVND',
  'B0C72RN927',
  'B0BLYPCR7F',
  'B0B192T3ZV'],
 'retrieved_context': ['jumper Laptop, 16 Inch FHD IPS Display (16:10), Intel Celeron Quad Core CPU, 4GB DDR4 128GB Storage, Windows 11 Laptops Computer with Office 365 1-Year Subscription, Numeric Keypad, 4 Stereo Speakers. 【16" Full-View HD Screen】Enjoy a vibrant visual experience on the 16 Inch Full HD IPS display with a resolution of 1920 x 1200 pixels. A stunning 16 : 10 widescreen display with 93% screen-to-body ratio gives you morespace to watch movies, edit photos, or brows the web. 【4-Core 4-Thread Powerful CPU】Jumper Laptop with the Intel Quad-core powerful processor, 4GB DDR4 RAM and 128GB Storage built-in, so experience smooth multita

### Reranking

In [ ]:
## Can work without api_key too.
cohere_client = cohere.ClientV2(api_key=os.environ["CO_API_KEY"])

In [39]:
to_rerank = results["retrieved_context"]

In [40]:
results

{'retrieved_context_ids': ['B0CGRSBNY1',
  'B0BVZ47N6N',
  'B09W9DV8PX',
  'B087ZMF5PR',
  'B0B87VXYFR',
  'B0BQFJVQ15',
  'B0944QNV3W',
  'B0B7XC9VL4',
  'B0BTKWY6XG',
  'B0C3RCQDM9',
  'B09QS7W8G5',
  'B0B8VH5WBR',
  'B0B46D4FM3',
  'B09T5YML6T',
  'B015TPLFFO',
  'B0CFLNNB4V',
  'B0BSGDCVND',
  'B0C72RN927',
  'B0BLYPCR7F',
  'B0B192T3ZV'],
 'retrieved_context': ['jumper Laptop, 16 Inch FHD IPS Display (16:10), Intel Celeron Quad Core CPU, 4GB DDR4 128GB Storage, Windows 11 Laptops Computer with Office 365 1-Year Subscription, Numeric Keypad, 4 Stereo Speakers. 【16" Full-View HD Screen】Enjoy a vibrant visual experience on the 16 Inch Full HD IPS display with a resolution of 1920 x 1200 pixels. A stunning 16 : 10 widescreen display with 93% screen-to-body ratio gives you morespace to watch movies, edit photos, or brows the web. 【4-Core 4-Thread Powerful CPU】Jumper Laptop with the Intel Quad-core powerful processor, 4GB DDR4 RAM and 128GB Storage built-in, so experience smooth multita

In [41]:
response = cohere_client.rerank(
    model="rerank-v4.0-pro",
    query=query,
    documents=to_rerank,
    top_n=20
)


In [31]:
response

V2RerankResponse(id='900523fd-7dfb-4977-b6a5-d0c7e2a4492f', results=[V2RerankResponseResultsItem(index=15, relevance_score=0.8442036), V2RerankResponseResultsItem(index=9, relevance_score=0.8314717), V2RerankResponseResultsItem(index=10, relevance_score=0.81908435), V2RerankResponseResultsItem(index=5, relevance_score=0.8126295), V2RerankResponseResultsItem(index=3, relevance_score=0.8053874), V2RerankResponseResultsItem(index=14, relevance_score=0.79027146), V2RerankResponseResultsItem(index=0, relevance_score=0.7863604), V2RerankResponseResultsItem(index=11, relevance_score=0.78239655), V2RerankResponseResultsItem(index=7, relevance_score=0.7743104), V2RerankResponseResultsItem(index=18, relevance_score=0.7715682), V2RerankResponseResultsItem(index=1, relevance_score=0.7103069), V2RerankResponseResultsItem(index=4, relevance_score=0.7103069), V2RerankResponseResultsItem(index=12, relevance_score=0.7005671), V2RerankResponseResultsItem(index=6, relevance_score=0.67542213), V2RerankRes

In [ ]:
## Could not figure this out?, Basically we orddered based on relevance score?
## It is creating a reranked_results list with right index ordered by small to large? <Confused>. I think we are using to_rerank to get the actual description.
## and following order of response.results.. Perfect
reranked_results = [to_rerank[result.index] for result in response.results]

In [33]:


reranked_results



['ACER 2023 Newest Swift X Laptop, 14" FHD Display, AMD Ryzen 5 5600U Processor (6 core), NVIDIA GeForce RTX 3050 Graphics, 8GB RAM, 1TB SSD, Backlit Keyboard, WiFi 6, Bluetooth, Windows 11 Home 【Processor】Equipped with AMD Ryzen 5 5600U, 2.3GHz (Up to Turbo Boost 4.2 GHz, 6 cores, 12 threads), this processor delivers exceptional performance that effortlessly handles demanding tasks. Experience seamless multitasking, productivity, and entertainment like never before. 【Graphics】 The laptop features a 14" FHD display. Equipped with NVIDIA GeForce RTX 3050 Graphics, making it ideal for gaming and content creation. Immerse yourself in smooth gaming experiences, create breathtaking graphics, and effortlessly handle resource-intensive tasks like video editing and 3D modeling. 【Upgraded】 Up to 8GB RAM is designed for basic tasks, the high-bandwith DDR4 RAM run your applications smoothly, as well as efficient gaming. Up to 1TB SSD storage capacity is suitable for saving all your files and prov

In [34]:

response = cohere_client.rerank(
    model="rerank-v4.0-fast",
    query=query,
    documents=to_rerank,
    top_n=20
)



In [35]:


reranked_results = [to_rerank[result.index] for result in response.results]



In [36]:


reranked_results



['Gateway 15.6" FHD Ultra Slim Laptop, Quad-Core Pentium Silver N5030 up to 3.1GHz, 4GB RAM, 128GB eMMC, WiFi, Bluetooth, Charcoal (Renewed) CPU -Pentium Silver N5030 Quad-Core Processor (1.10GHz, Up to 3.10GHz, 4M Cache) 15.6Inches FHD (1920 x 1200) LCD IPS Non-Touch Display; Integrated UHD Graphics 605. This electronic gift, holiday gift offers a good experience for watching videos, browsing the web, remote work, or study from home. 4GB RAM, 128GB eMMC Storage. No Optical Drive; 1MP Front-Facing Camera, Built-in Stereo Speakers x 2 and Built-in Microphone Ideal for Home, Student, Professionals, Small Business, School Education, and Commercial Enterprise, Online Class, Google Classroom, Remote Learning, Zoom Ready 802.11n WiFi and Bluetooth 4.0. HDMI x 1, USB Type-C x 1, USB 3.0 x 1, Micro SD Slot x 1, Kensington Lock x 1. Tuned by THX Audio, Built-in Stereo Speakers x 2, Built-in Microphone',
 'ACER 2023 Newest Swift X Laptop, 14" FHD Display, AMD Ryzen 5 5600U Processor (6 core), NV